# 股票分析看板代码

Below are the necessary code needed to run for generation of the Stock Analysis Dashboard, simply click Run All to see the dashboard at the bottom.

以下是运行股票分析看板的全部代码，只需要全部运行即可在页面底部使用。

## Import Necessary Packages

In [1]:
!pip install wbdata
!pip install ta
!pip install prophet

DEPRECATION: pyodbc 4.0.0-unsupported has a non-standard version number. pip 24.0 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of pyodbc or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063

[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: pip install --upgrade pip
DEPRECATION: pyodbc 4.0.0-unsupported has a non-standard version number. pip 24.0 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of pyodbc or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063

[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: pip install --upgrade pip
DEPRECATION: pyodbc 4.0.0-unsupported has a non-standard version number. pip 24.0 wi

In [2]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import yfinance as yf
from datetime import datetime, date
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
import altair as alt
import ipywidgets as widgets
from ipywidgets import VBox, HBox
from IPython.display import display, clear_output
from IPython.display import HTML
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.io as pio
import wbdata
# from statsmodels.tsa.arima.model import ARIMA
from prophet import Prophet
from prophet.diagnostics import cross_validation
from prophet.diagnostics import performance_metrics
from prophet.plot import plot_plotly, plot_components_plotly
import logging
from ipywidgets import Layout
import warnings
import ta  # Technical Analysis library for financial indicators

## 策略回测部分

In [3]:
def backtest_buy_and_hold_strategy(stock_data, investment, transaction_fee_percent):
    signals = pd.DataFrame(index=stock_data.index)
    signals['Close'] = stock_data['Close']
    
    signals['signal'] = 0.0
    signals['positions'] = 0.0
    
    signals.iloc[0, signals.columns.get_loc('signal')] = 1  # Buy on the first day
    signals.iloc[-1, signals.columns.get_loc('signal')] = -1  # Sell on the last day
    
    # Calculate transaction fees for buy and sell
    fee_multiplier_buy = (1 - transaction_fee_percent / 100)
    fee_multiplier_sell = (1 - transaction_fee_percent / 100)
    
    # Adjusting the investment for buy transaction fee
    adjusted_investment = investment * fee_multiplier_buy
    
    # Calculate portfolio value without considering sell fee
    signals['portfolio_value'] = adjusted_investment * (signals['Close'] / signals['Close'].iloc[0])
    
    # Apply sell transaction fee to the final value
    final_portfolio_value = signals['portfolio_value'].iloc[-1] * fee_multiplier_sell
    signals['portfolio_value'].iloc[-1] = final_portfolio_value
    
    signals['portfolio_returns'] = signals['Close'].pct_change()
    signals['positions'] = signals['signal']
    
    return signals

def backtest_ma_strategy(stock_data, investment, transaction_fee_percent):
    short_window = 50
    long_window = 200
    
    signals = pd.DataFrame(index=stock_data.index)
    signals['Close'] = stock_data['Close']
    
    # Calculate moving averages
    signals['short_mavg'] = stock_data['Close'].rolling(window=short_window, min_periods=1).mean()
    signals['long_mavg'] = stock_data['Close'].rolling(window=long_window, min_periods=1).mean()
    
    # Generate signals
    signals['signal'] = 0.0
    signals['signal'][short_window:] = np.where(signals['short_mavg'][short_window:] > signals['long_mavg'][short_window:], 1.0, 0.0)
    signals['positions'] = signals['signal'].diff()
    
    # Initialize portfolio value
    signals['portfolio_value'] = investment
    current_investment = investment
    last_buy_close = None  # This will track the last buy close price
    
    for index, row in signals.iterrows():
        if row['positions'] == 1.0:  # Buy
            buy_cost = current_investment * (transaction_fee_percent / 100)
            current_investment -= buy_cost  # Deduct buy transaction fee
            last_buy_close = row['Close']  # Update the last buy close price
        elif row['positions'] == -1.0 and last_buy_close is not None:  # Sell
            # Calculate sell revenue based on the last buy close price
            sell_revenue = current_investment * (row['Close'] / last_buy_close)
            sell_cost = sell_revenue * (transaction_fee_percent / 100)
            current_investment = sell_revenue - sell_cost  # Deduct sell transaction fee
        
        signals.at[index, 'portfolio_value'] = current_investment  # Update the current investment
    return signals

def backtest_rsi_strategy(stock_data, investment, transaction_fee_percent, lower_bound=30, upper_bound=70):
    signals = pd.DataFrame(index=stock_data.index)
    signals['Close'] = stock_data['Close']
    signals['RSI'] = ta.momentum.rsi(stock_data['Close'], window=14)

    # Initialize signals and positions
    signals['signal'] = 0.0
    signals['positions'] = 0.0
    is_position_open = False
    last_buy_price = None

    # Generate signals based on RSI
    for index, row in signals.iterrows():
        if row['RSI'] < lower_bound and not is_position_open:
            signals.at[index, 'signal'] = 1.0  # Buy signal
            signals.at[index, 'positions'] = 1.0  # Enter position
            is_position_open = True
            last_buy_price = row['Close']
        elif row['RSI'] > upper_bound and is_position_open:
            signals.at[index, 'signal'] = -1.0  # Sell signal
            signals.at[index, 'positions'] = -1.0  # Exit position
            is_position_open = False

    # Initialize portfolio value
    current_investment = investment
    signals['portfolio_value'] = investment  # Add portfolio_value column for compatibility
    
    # Assume each buy transaction uses the entire current investment to purchase shares
    for index, row in signals.iterrows():
        if row['positions'] == 1.0:  # Buy
            shares_to_buy = current_investment / (row['Close'] * (1 + transaction_fee_percent / 100))
            last_buy_price = row['Close']
            current_investment = shares_to_buy * row['Close']  # Update investment to reflect equity value
        elif row['positions'] == -1.0 and last_buy_price is not None:  # Sell
            sell_revenue = shares_to_buy * row['Close']  # Revenue from selling all shares
            sell_cost = sell_revenue * (transaction_fee_percent / 100)  # Transaction fee from selling
            current_investment = sell_revenue - sell_cost  # Update investment after selling
            last_buy_price = None  # Reset last buy price after selling

        signals.at[index, 'portfolio_value'] = current_investment
    return signals

def generate_transaction_history(signals):
    # Detect when transactions occur: a transaction is a change in the 'positions' column
    signals['prev_portfolio_value'] = signals['portfolio_value'].shift(1)
    transactions = signals[signals['positions'].notna() & (signals['positions'] != 0)].copy()
    
    # Reset index to ensure 'Date' is a column and not an index
    transactions.reset_index(inplace=True)
    
    # Determine the action based on the 'positions' column
    transactions['Action'] = transactions['positions'].apply(lambda x: 'Buy' if x > 0 else 'Sell')
    
    # Calculate the returns since the last transaction
    # For the first transaction, if calculating from the start, ensure you handle the edge case
    transactions['transaction_returns'] = transactions['portfolio_value'].pct_change().fillna(0)

    # Select and reorder relevant columns
    transactions = transactions[['Date', 'Action', 'positions', 'prev_portfolio_value', 'transaction_returns', 'portfolio_value']]
    
    # Round the transaction returns and portfolio values
    transactions['transaction_returns'] = transactions['transaction_returns'].round(4)
    transactions['portfolio_value'] = transactions['portfolio_value'].round(2)

    return transactions

def plot_backtest_results(signals, strategy='Buy and Hold', investment=10000):
    import plotly.graph_objects as go
    
    # Initialize the figure
    fig = go.Figure()

    # Plot the Close price
    fig.add_trace(go.Scatter(x=signals.index, y=signals['Close'], name='Close Price', line=dict(color='blue')))

    # Conditional plotting based on strategy
    if strategy in ['Moving Average Crossover', 'RSI Strategy']:
        if 'short_mavg' in signals:
            fig.add_trace(go.Scatter(x=signals.index, y=signals['short_mavg'], name='Short Moving Average', line=dict(color='red')))
        if 'long_mavg' in signals:
            fig.add_trace(go.Scatter(x=signals.index, y=signals['long_mavg'], name='Long Moving Average', line=dict(color='green')))

        # Plot Buy and Sell signals for strategies with these defined
        fig.add_trace(go.Scatter(x=signals[signals.positions == 1].index, 
                                 y=signals['Close'][signals.positions == 1],
                                 name='Buy Signal', mode='markers', marker_symbol='triangle-up', marker_line_color='green', marker_color='green', marker_size=10))
        fig.add_trace(go.Scatter(x=signals[signals.positions == -1].index, 
                                 y=signals['Close'][signals.positions == -1],
                                 name='Sell Signal', mode='markers', marker_symbol='triangle-down', marker_line_color='red', marker_color='red', marker_size=10))

    elif strategy == 'Buy and Hold':
        # For "Buy and Hold", mark the first and last data points
        fig.add_trace(go.Scatter(x=[signals.index[0]], 
                                 y=[signals['Close'].iloc[0]],
                                 name='Buy (Start)', mode='markers', marker_symbol='circle', marker_line_color='green', marker_color='green', marker_size=12))
        fig.add_trace(go.Scatter(x=[signals.index[-1]], 
                                 y=[signals['Close'].iloc[-1]],
                                 name='Sell (End)', mode='markers', marker_symbol='circle', marker_line_color='red', marker_color='red', marker_size=12))

    # Calculate total returns
    total_returns = signals['portfolio_value'].dropna().iloc[-1] - investment
    
    # Add an annotation for total returns
    fig.add_annotation(x=0.85, y=0.95, xref="paper", yref="paper",
                       text=f"Total Returns: {total_returns:.2f}",
                       showarrow=False, font=dict(size=14, color="green"),
                       bgcolor="white", bordercolor="black", borderwidth=1, borderpad=4)

    # Add titles and labels
    fig.update_layout(title=f'Backtest Result with {strategy}',
                      xaxis_title='Date',
                      yaxis_title='Price',
                      template="plotly_white")

    fig.show()
    
    transactions = generate_transaction_history(signals)

    # Create a table of transactions
    table = go.Figure(data=[go.Table(
        header=dict(values=['Date', 'Action', 'Positions', 'Previous Portfolio Value', 'Transaction Returns (%)', 'Portfolio Value'],
                    fill_color='lightsteelblue',
                    align='left'),
        cells=dict(values=[transactions['Date'].dt.strftime('%Y-%m-%d'), 
                           transactions['Action'],  
                           transactions['positions'],  
                           transactions['prev_portfolio_value'],  
                           (transactions['transaction_returns'] * 100).round(2).astype(str) + '%', 
                           transactions['portfolio_value'].round(2)],
                   fill_color='white',
                   align='left'))
    ])

    table.update_layout(title='Transaction History', width=800, height=400, margin=dict(l=20, r=20, t=40, b=20))
    table.show()


## 看板主要部分

In [4]:
cmdstanpy_logger = logging.getLogger("cmdstanpy")
cmdstanpy_logger.disabled = True

# Function to download stock data
def download_stock_data(ticker, start_date, end_date):
    stock_data = yf.download(ticker, start=start_date, end=end_date, progress=False)
    stock_data_with_indicators = add_technical_indicators(stock_data)
    return stock_data_with_indicators

# Function to plot stock data
def plot_stock_data(stock_1_data, stock_1_ticker, stock_2_data=None, stock_2_ticker=None, plot_indicators=[]):
    # Create a subplot that allows for a secondary y-axis
    fig = make_subplots(specs=[[{"secondary_y": True}]])

    # Plot the close prices for the first stock
    fig.add_trace(
        go.Scatter(
            x=stock_1_data.index,
            y=stock_1_data['Close'],
            mode='lines',  # Adds markers to the steps for better visibility
            name=f'{stock_1_ticker} Close Price',
            line=dict(color='#1f77b4', shape='hv'),  # 'hv' creates a step-like line
        ),
        secondary_y=False,  # Primary Y-axis
    )

    # Example of adding technical indicators to the plot
    # Moving Averages
    if 'MA50' in plot_indicators:
        fig.add_trace(go.Scatter(x=stock_1_data.index, y=stock_1_data['MA50'], mode='lines', name='MA50', line=dict(color='#e377c2')), secondary_y=False)
    if 'MA200' in plot_indicators:
        fig.add_trace(go.Scatter(x=stock_1_data.index, y=stock_1_data['MA200'], mode='lines', name='MA200', line=dict(color='#8c564b')), secondary_y=False)

    # RSI - Plotted on the secondary y-axis
    if 'RSI' in plot_indicators:
        fig.add_trace(go.Scatter(x=stock_1_data.index, y=stock_1_data['RSI'], mode='lines', name='RSI', line=dict(color='purple')), secondary_y=True)

    # Bollinger Bands
    if 'Bollinger_High' in plot_indicators and 'Bollinger_Low' in plot_indicators:
        fig.add_trace(go.Scatter(x=stock_1_data.index, y=stock_1_data['Bollinger_High'], mode='lines', name='Bollinger High', line=dict(color='green')), secondary_y=False)
        fig.add_trace(go.Scatter(x=stock_1_data.index, y=stock_1_data['Bollinger_Low'], mode='lines', name='Bollinger Low', line=dict(color='red')), secondary_y=False)

    # MACD
    if 'MACD' in plot_indicators and 'MACD_Signal' in plot_indicators:
        fig.add_trace(go.Scatter(x=stock_1_data.index, y=stock_1_data['MACD'], mode='lines', name='MACD', line=dict(color='#FF00FF')), secondary_y=True)
        fig.add_trace(go.Scatter(x=stock_1_data.index, y=stock_1_data['MACD_Signal'], mode='lines', name='MACD Signal', line=dict(color='#0000FF')), secondary_y=True)
    
    # Ichimoku Cloud
    if 'Ichimoku_A' in plot_indicators:
        fig.add_trace(go.Scatter(x=stock_1_data.index, y=stock_1_data['Ichimoku_A'], mode='lines', name='Ichimoku A', line=dict(color='#00FF00')), secondary_y=False)
    if 'Ichimoku_B' in plot_indicators:
        fig.add_trace(go.Scatter(x=stock_1_data.index, y=stock_1_data['Ichimoku_B'], mode='lines', name='Ichimoku B', line=dict(color='#FF0000')), secondary_y=False)

    # Plot the close prices for the second stock, if provided
    if stock_2_data is not None and stock_2_ticker:
        fig.add_trace(
            go.Scatter(
                x=stock_2_data.index,
                y=stock_2_data['Close'],
                mode='lines',  # Adds markers to the steps for better visibility
                name=f'{stock_2_ticker} Close Price',
                line=dict(color='#ff7f0e', shape='hv'),  # 'hv' for a step-like line
            ),
            secondary_y=True,  # Decide based on your preference
        )

    # Update layout with titles and axis labels
    fig.update_layout(
        title="Stock Close Prices and Technical Indicators",
        xaxis_title="Date",
        yaxis_title="Price",
        template="plotly_white"
    )

    fig.show()
    
# Function to display analysis results
def display_results(stock_ticker_1, stock_ticker_2, correlation, beta, volatility_1, volatility_2):
    results_html = f"""
    <table>
    <tr><th>Analysis</th><th>{stock_ticker_1}</th><th>{stock_ticker_2}</th></tr>
    <tr><td>Pearson Correlation between {stock_ticker_1} and {stock_ticker_2}</td><td colspan='2'>{correlation:.4f}</td></tr>
    <tr><td>Beta Coefficient of {stock_ticker_1} relative to {stock_ticker_2}</td><td colspan='2'>{beta:.4f}</td></tr>
    <tr><td>Volatility</td><td>{volatility_1:.4f}</td><td>{volatility_2:.4f}</td></tr>
    </table>
    """
    display(HTML(results_html))

# Function for financial analysis
def perform_financial_analysis(stock_1_data, stock_1_ticker, stock_2_data, stock_2_ticker):
    aligned_data = pd.merge(stock_1_data.reset_index(), stock_2_data.reset_index(), on='Date', suffixes=('_1', '_2'))

    # Assuming the 'Close' columns are named 'Close_1' and 'Close_2' after merging
    close_1_column = 'Close_1'
    close_2_column = 'Close_2'

    # Pearson Correlation
    correlation = aligned_data[close_1_column].corr(aligned_data[close_2_column])
    
    # Beta Coefficient
    covariance = np.cov(aligned_data[close_1_column].pct_change().dropna(), aligned_data[close_2_column].pct_change().dropna())[0][1]
    variance = aligned_data[close_2_column].pct_change().var()
    beta = covariance / variance
    
    # Volatility Comparison
    volatility_1 = aligned_data[close_1_column].pct_change().std()
    volatility_2 = aligned_data[close_2_column].pct_change().std()
    
    # Display results
    display_results(stock_1_ticker, stock_2_ticker, correlation, beta, volatility_1, volatility_2)
    
# New function for prediction
def predict_stock_price(stock_data, periods=30):
    df = stock_data.reset_index()
    df.rename(columns={'Date': 'ds', 'Close': 'y'}, inplace=True)  # Assuming 'Close' is the column with stock prices
    
    model = Prophet()
    model.fit(df)
    
    future = model.make_future_dataframe(periods=periods)
    forecast = model.predict(future)
    
    return forecast, model
    
# Function to display prediction properties
def display_prediction_properties(model, forecast):
    fig = make_subplots(rows=3, cols=1)
    # Trend
    fig.add_trace(go.Scatter(x=forecast['ds'], y=forecast['trend'], name='Trend'), row=1, col=1)
    
    # Yearly
    fig.add_trace(go.Scatter(x=forecast['ds'], y=forecast['yearly'], name='Yearly Seasonality'), row=2, col=1)
    
    # Weekly
    if 'weekly' in forecast:
        fig.add_trace(go.Scatter(x=forecast['ds'], y=forecast['weekly'], name='Weekly Seasonality'), row=3, col=1)
    
    fig.update_layout(height=600, width=800, title_text="Prophet Model Components")
    fig.show()
    
def evaluate_prophet_model(model, df, start_date, end_date):
    # Calculate the total days of data available
    total_days = (end_date - start_date).days
    
    # Ensure there's enough data for initial training period and horizon
    if total_days <= 730:  # Less than or equal to 2 years
        print("Limited data available for evaluation. Adjusting validation strategy.")
        # Adjust 'initial' and 'horizon' based on available data
        initial_days = max(180, total_days // 3)  # Use a minimum of 6 months or a third of total days
        horizon_days = max(30, total_days // 6)  # Use a minimum of 1 month or a sixth of total days
    else:
        # Set 'initial' to half of the data's timeframe up to a maximum of 3 years (in days)
        initial_days = min(1095, total_days // 2)  # 3 years in days or half of the total days
        horizon_days = 365  # Set 'horizon' to 1 year (365 days)
    
    initial = f'{initial_days} days'
    horizon = f'{horizon_days} days'
    
    # Set 'period' based on the adjusted 'initial' period to ensure multiple evaluations
    period_days = max(90, initial_days // 4)  # At least 3 months, a quarter of the initial
    period = f'{period_days} days'
    
    try:
        # Perform cross-validation
        df_cv = cross_validation(model, initial=initial, period=period, horizon=horizon)
        
        # Calculate performance metrics
        df_p = performance_metrics(df_cv)
        
        # Display evaluation metrics
        display_evaluation_metrics(df_p)
    except Exception as e:
        print(f"Error during model evaluation: {e}")

def display_evaluation_metrics(df_p):
    metrics = df_p.mean().to_dict()  # Averaging performance metrics for simplicity
    
    metrics_html = "<div><h3>Model Evaluation Metrics</h3><table border='1'><tr>"
    metrics_html += "".join([f"<th>{metric}</th>" for metric in metrics.keys()])
    metrics_html += "</tr><tr>"
    
    # Adjust formatting to handle Timedelta and numeric values separately
    for value in metrics.values():
        if isinstance(value, pd.Timedelta):
            # Convert Timedelta to days, for example
            formatted_value = f"{value.total_seconds() / 86400:.2f} days"
        else:
            # Assume value is numeric and format as before
            formatted_value = f"{value:.4f}"
        metrics_html += f"<td>{formatted_value}</td>"
    
    metrics_html += "</tr></table></div>"
    display(HTML(metrics_html))
    
# Button click event handler for prediction
def on_predict_button_clicked(b):
    with output:
        clear_output(wait=True)
        start_date_val = pd.to_datetime(start_date.value)
        end_date_val = pd.to_datetime(end_date.value)
        stock_data = download_stock_data(stock_ticker_1.value, start_date_val, end_date_val)
        forecast, model = predict_stock_price(stock_data, periods=30)
        
        # Use Plotly for interactive forecast plot
        fig = plot_plotly(model, forecast)
        fig.update_layout(title="Stock Price Forecast")
        fig.show()

        # Optionally, use Plotly for interactive components plot
        fig_components = plot_components_plotly(model, forecast)
        fig_components.update_layout(title="Prophet Model Components")
        fig_components.show()
        
        # Evaluate and display metrics horizontally
        evaluate_prophet_model(model, stock_data, start_date_val, end_date_val)
        
def add_technical_indicators(stock_data):
    # Ichimoku Cloud
    ichimoku = ta.trend.IchimokuIndicator(high=stock_data['High'], low=stock_data['Low'], window1=9, window2=26, window3=52)
    stock_data['Ichimoku_A'] = ichimoku.ichimoku_a()
    stock_data['Ichimoku_B'] = ichimoku.ichimoku_b()
    
    # Calculate Moving Averages
    stock_data['MA50'] = ta.trend.sma_indicator(stock_data['Close'], window=50)
    stock_data['MA200'] = ta.trend.sma_indicator(stock_data['Close'], window=200)
    
    # Calculate RSI
    stock_data['RSI'] = ta.momentum.rsi(stock_data['Close'], window=14)
    
    # Calculate Bollinger Bands
    bollinger_bands = ta.volatility.BollingerBands(stock_data['Close'])
    stock_data['Bollinger_High'] = bollinger_bands.bollinger_hband()
    stock_data['Bollinger_Low'] = bollinger_bands.bollinger_lband()
    
    # Calculate MACD
    macd = ta.trend.MACD(stock_data['Close'])
    stock_data['MACD'] = macd.macd()
    stock_data['MACD_Signal'] = macd.macd_signal()
    
    return stock_data        

# The main function to fetch data, plot, and calculate financial metrics
def fetch_plot_correlation(stock_ticker_1, stock_ticker_2, start_date, end_date, plot_indicators=[]):
    stock_1_data = download_stock_data(stock_ticker_1, start_date, end_date)
    stock_2_data = download_stock_data(stock_ticker_2, start_date, end_date) if stock_ticker_2 else None
    
    plot_stock_data(stock_1_data, stock_ticker_1, stock_2_data, stock_ticker_2, plot_indicators)
    
    if stock_ticker_2:
        perform_financial_analysis(stock_1_data, stock_ticker_1, stock_2_data, stock_ticker_2)
        
    
# Define the button click event handler
def on_button_clicked(b):
    with output:
        clear_output(wait=True)  # Clear the output before displaying new content
        # Determine if the second ticker should be considered based on the mode
        ticker_2 = stock_ticker_2.value if analysis_mode.value == 'Multiple Stock' else None
        transaction_fee = transaction_fee_percent.value if analysis_mode.value == 'Backtesting' else None
        # In "Single Stock" mode, consider selected indicators; otherwise, bypass technical analysis
        if analysis_mode.value == 'Single Stock':
            selected_indicators = []
            # Check for existing indicator selections
            if checkbox_MA50.value: selected_indicators.append('MA50')
            if checkbox_MA200.value: selected_indicators.append('MA200')
            if checkbox_RSI.value: selected_indicators.append('RSI')
            if checkbox_BB.value: selected_indicators.extend(['Bollinger_High', 'Bollinger_Low'])
            if checkbox_MACD.value: selected_indicators.extend(['MACD', 'MACD_Signal'])
            if checkbox_Ichimoku.value: selected_indicators.extend(['Ichimoku_A', 'Ichimoku_B'])
            fetch_plot_correlation(stock_ticker_1.value, None, start_date.value.strftime('%Y-%m-%d'), end_date.value.strftime('%Y-%m-%d'), selected_indicators)
        elif analysis_mode.value == 'Multiple Stock':
            # In "Multiple Stock" mode, bypass technical indicators and proceed without them
            fetch_plot_correlation(stock_ticker_1.value, ticker_2, start_date.value.strftime('%Y-%m-%d'), end_date.value.strftime('%Y-%m-%d'))
        elif analysis_mode.value == 'Backtesting':
            start_date_val = pd.to_datetime(start_date.value)
            end_date_val = pd.to_datetime(end_date.value)
            stock_data = download_stock_data(stock_ticker_1.value, start_date_val, end_date_val)

            # Check the selected strategy and apply the transaction fee
            if strategy_selection.value == 'Moving Average Crossover':
                backtest_results = backtest_ma_strategy(stock_data, investment_amount.value, transaction_fee)
                plot_backtest_results(backtest_results, 'Moving Average Crossover', investment_amount.value)
            elif strategy_selection.value == 'RSI Strategy':
                backtest_results = backtest_rsi_strategy(stock_data, investment_amount.value, transaction_fee)
                plot_backtest_results(backtest_results, 'RSI Strategy', investment_amount.value)
            elif strategy_selection.value == 'Buy and Hold':
                backtest_results = backtest_buy_and_hold_strategy(stock_data, investment_amount.value, transaction_fee)
                plot_backtest_results(backtest_results, 'Buy and Hold', investment_amount.value)

# Layout adjustment based on the analysis mode
def adjust_layout(*args):
    if analysis_mode.value == 'Single Stock':
        stock_ticker_2.layout.display = 'none'
        predict_button.layout.display = 'flex'
        technical_indicators_widgets.layout.display = 'block'
        backtesting_widgets.layout.display = 'none'
    elif analysis_mode.value == 'Multiple Stock':
        stock_ticker_2.layout.display = 'flex'
        predict_button.layout.display = 'none'
        technical_indicators_widgets.layout.display = 'none'
        backtesting_widgets.layout.display = 'none'
    elif analysis_mode.value == 'Backtesting':
        stock_ticker_2.layout.display = 'none'  # Hide the second ticker input for backtesting
        predict_button.layout.display = 'none'
        technical_indicators_widgets.layout.display = 'none'
        backtesting_widgets.layout.display = 'flex'
        
# UI Components
# Main Components
dashboard_heading = widgets.HTML(value="<h2 style='text-align: left; color: #c71306;'>股票分析看板</h2><p style='font-size: 12px; color: #808080;'>仅供娱乐和教育使用; Entertainment & Educational Usage Only</p>")
analysis_mode = widgets.Dropdown(options=['Single Stock', 'Multiple Stock', 'Backtesting'], value='Single Stock', description='Mode:')
stock_ticker_1 = widgets.Text(value='000001.SS', description='Ticker 1:', disabled=False)
stock_ticker_2 = widgets.Text(value='^HSI', description='Ticker 2:', disabled=False)
start_date = widgets.DatePicker(description='Start Date', value=pd.to_datetime('2010-01-01'))
end_date = widgets.DatePicker(description='End Date', value=pd.to_datetime(datetime.now().strftime('%Y-%m-%d')))
run_button = widgets.Button(
    description='Analyze',
    layout=Layout(width='auto', height='auto', justify_content='center'))
predict_button = widgets.Button(
    description='Time Series Analysis',
    layout=Layout(width='auto', height='auto', justify_content='center')
)
predict_button.layout.display = 'none'  # Initially hidden

# Technical Analysis
checkbox_Ichimoku = widgets.Checkbox(value=False, description='Ichimoku Cloud')
checkbox_Fibonacci = widgets.Checkbox(value=False, description='Fibonacci Retracements')
checkbox_MA50 = widgets.Checkbox(value=False, description='MA50')
checkbox_MA200 = widgets.Checkbox(value=False, description='MA200')
checkbox_RSI = widgets.Checkbox(value=False, description='RSI')
checkbox_BB = widgets.Checkbox(value=False, description='Bollinger Bands')
checkbox_MACD = widgets.Checkbox(value=False, description='MACD')

# Backtesting 
strategy_selection = widgets.Dropdown(
    options=['Buy and Hold', 'Moving Average Crossover', 'RSI Strategy'],
    value='Buy and Hold',
    description='策略:',
    disabled=False,
)

investment_amount = widgets.FloatText(
    value=10000,
    description='本金:',
    disabled=False
)
transaction_fee_percent = widgets.FloatText(
    value=0.3,  # Default to 0.1% transaction fee; adjust as needed
    description='交易费率(%):',
    disabled=False
)

# Output
output = widgets.Output()

# Grouping stock selection widgets
stock_selection_widgets = VBox([dashboard_heading,
                                HBox([stock_ticker_1, stock_ticker_2]),
                                analysis_mode])

# Grouping date selection widgets
date_selection_widgets = HBox([start_date, end_date])

# Grouping technical indicators checkboxes
technical_indicators_widgets = VBox([checkbox_MA50, 
                                      checkbox_MA200, 
                                      checkbox_RSI, 
                                      checkbox_BB, 
                                      checkbox_MACD,
                                      checkbox_Ichimoku])

# Grouping action buttons
action_buttons = HBox([run_button, predict_button])

# Grouping backtesting widgets
backtesting_widgets = VBox([strategy_selection, investment_amount, transaction_fee_percent])

# Final display layout
dashboard_layout = VBox([stock_selection_widgets,
                         date_selection_widgets,
                         technical_indicators_widgets,
                         backtesting_widgets,  # Include this line
                         action_buttons,
                         output])

predict_button.on_click(on_predict_button_clicked)
run_button.on_click(on_button_clicked)

analysis_mode.observe(adjust_layout, 'value')
adjust_layout()  # Adjust layout initially based on the default mode

display(dashboard_layout)
